In [ ]:

import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plots
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import io
import os
import random
#from google.colab import files
from PIL import Image
import albumentations as A
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, InputLayer, BatchNormalization, Input, Dropout, RandomFlip, RandomRotation, Resizing, Rescaling
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TruePositives, TrueNegatives, Precision, Recall, AUC, binary_accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, CSVLogger, EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers  import L2, L1
from tensorboard.plugins.hparams import api as hp
#from google.colab import drive

: 

In [ ]:
import matplotlib.pyplot as plt


: 

In [ ]:
pip install --upgrade tensorflow-datasets

: 

In [ ]:
dataset, dataset_info =tfds.load('malaria', with_info = True, as_supervised =True, shuffle_files=True, split=['train'])

: 

In [ ]:
 def split(dataset,TRAIN_RATIO,VAL_RATIO,TEST_RATIO):

    DATASET_SIZE = len(dataset)
    train_dataset= dataset.take(int(TRAIN_RATIO * DATASET_SIZE))

    val_test_dataset= dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
    val_dataset=val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

    test_dataset=val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))

    return train_dataset, val_dataset, test_dataset

: 

In [ ]:
TRAIN_RATIO= 0.8
VAL_RATIO= 0.1
TEST_RATIO=0.1

#dataset= tf.data.Dataset.range(10)
train_dataset, val_dataset, test_dataset= split(dataset[0],TRAIN_RATIO,VAL_RATIO,TEST_RATIO)
print(train_dataset, val_dataset, test_dataset)
print(list(train_dataset.take(1).as_numpy_iterator()), list(val_dataset.take(1).as_numpy_iterator()),list(test_dataset.take(1).as_numpy_iterator()))


: 

**Data Visualisation **

In [ ]:
for i, (image,label) in enumerate (train_dataset.take(16)):
  ax = plt.subplot(4,4,i+1)
  plt.imshow(image)
  plt.title(dataset_info.features['label'].int2str(label) )
  plt.axis('off')
dataset_info.features['label'].int2str(1)


: 

**Data Processing**

In [ ]:
IM_SIZE = 224
def resize_rescale(image,label):
  return tf.image.resize(image, (IM_SIZE, IM_SIZE))/255.0, label

: 

In [ ]:
train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)
train_dataset

for image,label in train_dataset.take(1):
  print(image,label)

: 

In [ ]:
BATCH_SIZE = 32
train_dataset= train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)


: 

In [ ]:
#BATCH_SIZE = 32
val_dataset= val_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)



: 

In [ ]:
#BATCH_SIZE = 32
test_dataset

: 

Model Creation  

In [ ]:
lenet_model = tf.keras.Sequential([
    InputLayer(input_shape = (IM_SIZE,IM_SIZE,3)),
    Conv2D(filters = 6, kernel_size =3, strides=1, padding = 'valid',activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2, strides =2 ),

    Conv2D(filters = 16, kernel_size =3, strides=(1,1), padding = 'valid',activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size= 2, strides =2),

    Flatten(),

    Dense(100, activation = "relu"),
    BatchNormalization(),

    Dense(10, activation = "relu"),
    BatchNormalization(),

    Dense(1, activation = "sigmoid"),


])
lenet_model.summary()


: 

In [ ]:
import tensorflow as tf

# Define the true labels and predicted values
y_true = tf.constant([0,1,2,3])  # Explicitly convert to TensorFlow tensor
y_pred = tf.constant([0.2,0.5,0.94,1])  # Explicitly convert to TensorFlow tensor

# Initialize the BinaryCrossentropy loss function
bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)

# Calculate the loss
loss = bce(y_true, y_pred)
print("Binary Crossentropy Loss:", loss.numpy())

: 

In [ ]:
lenet_model.compile(optimizer= Adam(learning_rate =0.1),
  loss =BinaryCrossentropy(),
  metrics=['accuracy'])

history = lenet_model.fit(train_dataset,validation_data=val_dataset,epochs=20, verbose =1)



: 

In [ ]:
history = lenet_model.fit(train_dataset,validation_data=val_dataset,epochs=20, verbose =1)


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train_loss','val_loss'])
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('epoch')
plt.ylabel('Accuracy')
plt.legend(['train_accuracy','val_accuracy'])
plt.show()

: 

**Model Evaluating and Testing**

In [ ]:
lenet_model.evaluate()

: 